In [1]:
import sys
import os
from cli_new import HuatuoChatbot
import torch
from PIL import Image
import json
from tqdm import tqdm
import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer
import torch
import json
from tqdm import tqdm
import re
import copy



import seaborn as sns
import matplotlib.pyplot as plt

from methods import *


NUM_IMG_TOKENS = 576
PATCHES = 24
SIZE = (336, 336)

bot = HuatuoChatbot("FreedomIntelligence/HuatuoGPT-Vision-7B", device="cuda:0")

/home/user/Desktop/code/HuatuoGPT-Vision
loading from FreedomIntelligence/HuatuoGPT-Vision-7B
[2025-05-10 19:18:15,605] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/user/Downloads/ENTER/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


loading from CLIP first. This should only be used at inference!!!
loading vision model from openai/clip-vit-large-patch14-336


/home/user/Downloads/ENTER/envs/llava/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import numpy as np

def get_GT_KnockOut_acc(json_path, output_filename, filename_suffix, masked_layer):

    if json_path == "slake_qa_set": # skip this, because SLAKE QA has no gt_token
        return
    
    qa_set_attention_maps = np.load(f'./attention_maps/{output_filename}_set_attention_maps{filename_suffix}.npy', allow_pickle=True).item()

    correct = 0
    total = 0
    incorrect_ids_qa_ak=[]

    for id, qa in tqdm(qa_set_attention_maps.items()):
        prompt = f"{qa['question']} Answer the question using a single word or phrase."
        
        # 2nd pass
        # inputs_embeds_shape = input_ids.shape[-1] - 1 + NUM_IMG_TOKENS
        input_ids = np.array(qa["input_ids"])
        image_start_token_index = np.where(input_ids==-200)[0][0]
        gt_tokens = qa["gt_token"]
        gt_tokens = [token_index[0]*PATCHES + token_index[1] for token_index in gt_tokens]
        all_img_tokens = [idx for idx in range(NUM_IMG_TOKENS)]
        inv_gt_tokens = [token for token in all_img_tokens if token not in gt_tokens]
        
        last_token_index = [idx for idx in range(image_start_token_index + NUM_IMG_TOKENS, input_ids.shape[-1] + NUM_IMG_TOKENS - 1)]
        if not isinstance(last_token_index, list):
            last_token_index = [last_token_index]

        block_config = {}
        image_range = [image_start_token_index + patch_index for patch_index in inv_gt_tokens]

        block_ids = [image_range, last_token_index]
        temp = [(stok1, stok0) for stok0 in block_ids[0] for stok1 in block_ids[1]]

        for layer in masked_layer:
            block_config[layer] = copy.deepcopy(temp)

        with torch.inference_mode():
            block_attn_hooks = set_block_attn_hooks_llava(bot.model, block_config, block_desc="Image->Question")
            _, input_ids, response_qs = bot.inference(prompt, qa['image_path'])
            remove_wrapper_llava(bot.model, block_attn_hooks)

        answer = qa['answer']

        clean_response = response_qs.replace("<|endoftext|>", "")

        if clean_response in answer or answer in clean_response:
            correct += 1
        else:
            incorrect_ids_qa_ak.append(id)
        total += 1

    print(f"GT Accuracy on {json_path.replace('_', ' ')}: {(correct / total) * 100:.4f}%")

    os.makedirs("./incorrect", exist_ok=True)
    with open(f"./incorrect/incorrect_ids_{output_filename}_gt{filename_suffix}.json", "w") as f:
        json.dump(incorrect_ids_qa_ak, f)

In [2]:
import json
import random
import copy
import cv2
import numpy as np


def convert_bounding_box_to_resized(bboxes, original_size, new_size):
    # Extract original image size and resized image size
    W_orig, H_orig = original_size
    W_new, H_new = new_size

    # Compute scaling factors
    scale_x = W_new / W_orig
    scale_y = H_new / H_orig

    # Handle both single bbox and list of bboxes
    if not isinstance(bboxes[0], list) and not isinstance(bboxes[0], tuple):
        bboxes = [bboxes]
    
    resized_bboxes = []
    for bbox in bboxes:
        # Extract the bounding box in the original image
        x_min, y_min, width, height = bbox

        # Convert to new image coordinates
        x_min_new = x_min * scale_x
        y_min_new = y_min * scale_y
        width_new = width * scale_x
        height_new = height * scale_y

        resized_bboxes.append([x_min_new, y_min_new, width_new, height_new])
    
    return resized_bboxes


def get_tokens_covering_bbox(bboxes, num_tokens=24):

    img_width, img_height = SIZE
    token_width = img_width / num_tokens
    token_height = img_height / num_tokens
    
    # Convert single bbox to list for uniform processing
    if not isinstance(bboxes, list):
        bboxes = [bboxes]
    
    selected_tokens = set()  # Use set to avoid duplicate tokens
    
    for bbox in bboxes:
        x_min, y_min, width, height = bbox
        
        x_min_token = int(np.floor(x_min / token_width))
        y_min_token = int(np.floor(y_min / token_height))
        x_max_token = int(np.ceil((x_min + width) / token_width))
        y_max_token = int(np.ceil((y_min + height) / token_height))

        x_max_token = min(x_max_token, num_tokens - 1)
        y_max_token = min(y_max_token, num_tokens - 1)
        
        ### Note it is (y, x) here
        bbox_tokens = [(y, x) for x in range(x_min_token, x_max_token) 
                               for y in range(y_min_token, y_max_token)]
        selected_tokens.update(bbox_tokens)

    return list(selected_tokens)

def draw_tokens_on_image(img, token_indices, num_tokens=24, color=(0, 255, 0), thickness=1):
    # Load the image
    h, w = SIZE  # Get image dimensions

    # Compute grid cell size
    token_width = w / num_tokens
    token_height = h / num_tokens
    
    # Create a copy of the image if it's a tensor
    if isinstance(img, torch.Tensor):
        img_np = img.numpy().copy()
    else:
        img_np = np.array(img).copy()
    
    # Make sure the image is in the right format for OpenCV
    if len(img_np.shape) == 2:  # If it's a grayscale image
        img_np = cv2.cvtColor(img_np.astype(np.float32), cv2.COLOR_GRAY2BGR)
    
    # Convert to uint8 if it's float
    if img_np.dtype == np.float32 or img_np.dtype == np.float64:
        img_np = (img_np * 255).astype(np.uint8)

    # Already switch x and y here
    for y_token, x_token in token_indices:
        # Compute the top-left corner pixel coordinates of the token
        x_pixel = int(x_token * token_width)
        y_pixel = int(y_token * token_height)

        # Draw the rectangle (top-left, bottom-right)
        cv2.rectangle(img_np, (x_pixel, y_pixel),
                      (int((x_token + 1) * token_width), int((y_token + 1) * token_height)),
                      color, thickness)

    return img_np

def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

def show_mask_on_image(img, mask):
    img = np.float32(img) / 255
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_HSV)
    hm = np.float32(heatmap) / 255
    cam = hm + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam), heatmap

def visualize_attention_map(att_map, image_path, gt_token=None, threshold=None, mask=None, v_minmax=None, save_image=False, id=None):

    att_map_scaled = torch.tensor(att_map)
    att_map_scaled = normalize(att_map_scaled)

    # To only show the bbox area
    if mask and (gt_token is not None):
        mask = np.zeros_like(att_map_scaled)
        for x, y in gt_token:
            mask[x, y] = 1

        att_map_scaled = att_map_scaled * mask

    attn_over_image = torch.nn.functional.interpolate(
        att_map_scaled.unsqueeze(0).unsqueeze(0), 
        size=(336,336), 
        mode='nearest', 
    ).squeeze()

    img = Image.open(image_path).resize((336, 336))
    np_img = np.array(img)
    img_with_attn, _ = show_mask_on_image(np_img, attn_over_image.numpy())

    if gt_token is not None:
        np_img = draw_tokens_on_image(np_img, gt_token)

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    axes[0].imshow(np_img)
    axes[0].axis("off")

    if v_minmax is not None:
        axes[1].imshow(att_map_scaled, vmin=v_minmax[0], vmax=v_minmax[1])
    else:
        axes[1].imshow(att_map_scaled)
        
    axes[1].axis("off")

    axes[2].imshow(img_with_attn)
    axes[2].axis("off")

    plt.tight_layout()
    plt.show()

    if save_image:
        sample_id = image_path.split("/")[-2]
        # Create numbered output directory
        output_dir = os.path.join("./output_images", sample_id, f"qid{id}")
        os.makedirs(output_dir, exist_ok=True)
        
        if v_minmax is not None:
            output_filename = os.path.join(output_dir, "after.png")
        else:
            output_filename = os.path.join(output_dir, "before.png")

        fig.savefig(output_filename, bbox_inches='tight', dpi=300)

        original_img_path = os.path.join(output_dir, "source.png")
        img.save(original_img_path)

        att_map_path = output_filename.replace(".png", ".npy")
        np.save(att_map_path, att_map_scaled.numpy())

        # Save the attention map
        plt.figure(figsize=(4, 4))
        if v_minmax is not None:
            plt.imshow(att_map_scaled, vmin=v_minmax[0], vmax=v_minmax[1])
        else:
            plt.imshow(att_map_scaled)
        plt.axis("off")
        att_map_img_path = output_filename.replace(".png", "_att.png")
        plt.savefig(att_map_img_path, bbox_inches='tight', dpi=300)
        plt.close()
        
    if v_minmax is None:
        v_minmax = (att_map_scaled.min(), att_map_scaled.max())
        return v_minmax
    
def attention_ratio(att_map, gt_tokens):
    relevant_attention = sum([att_map[tuple(token)] for token in gt_tokens])
    average_attention = att_map.sum() / NUM_IMG_TOKENS * len(gt_tokens)
    return relevant_attention / (average_attention + 1e-8)


## Masked Attention

In [3]:
def model_inference(image_path, question=None):
    if not question:
        question = 'Write a general description of the image. Answer the question using a single word or phrase.'
    model_output, input_ids, response_qs = bot.inference(question, image_path, return_att_map=True)
    return model_output, input_ids, response_qs

def get_attention_map(model_output, input_ids, layers, Q=False, layer_heads=None):
    input_ids = input_ids[0].cpu()
    index = torch.where(input_ids==-200)[0]
    
    att_maps = {}

    if layer_heads is not None:
        for layer_head in layer_heads:
            layer, head = layer_head[0], layer_head[1]
            if Q:
                att_maps[(layer, head)] = model_output['attentions'][layer][0, head, index+NUM_IMG_TOKENS:, index:index+NUM_IMG_TOKENS].mean(dim=(0)).to(torch.float32).detach().cpu().numpy().reshape(PATCHES, PATCHES)
            else:
                att_maps[(layer, head)] = model_output['attentions'][layer][0, head, -1, index:index+NUM_IMG_TOKENS].to(torch.float32).detach().cpu().numpy().reshape(PATCHES, PATCHES)
    else:
        for layer in layers:
            if Q:
                att_maps[layer] = model_output['attentions'][layer][0, :, index+NUM_IMG_TOKENS:, index:index+NUM_IMG_TOKENS].mean(dim=(0,1)).to(torch.float32).detach().cpu().numpy().reshape(PATCHES, PATCHES)

            else:
                att_maps[layer] = model_output['attentions'][layer][0, :, -1, index:index+NUM_IMG_TOKENS].mean(dim=0).to(torch.float32).detach().cpu().numpy().reshape(PATCHES, PATCHES)
        
    return att_maps

In [4]:
import numpy as np

def get_Attention_Maps(json_path, output_filename, Q=True, norm=True, layer_heads=None, filename_suffix=""):

    with open(f'{json_path}.json', 'r') as f:
        qa_set = json.load(f)

    qa_set_attention_maps = {}
    for qa in tqdm(qa_set):
        id = qa['qid']
        prompt = f"{qa['question']} Answer the question using a single word or phrase."
        image_path = os.path.join(data_path, qa['image_name'])

        # 1st pass
        model_output, input_ids, _ = model_inference(image_path, prompt)
        att_maps = get_attention_map(model_output, input_ids, layers=[l for l in range(28)], Q=Q, layer_heads=layer_heads)

        if norm:
            model_general_output, input_general_ids, _ = model_inference(image_path)
            # general_att_maps = get_attention_map(model_general_output, input_general_ids, layers=[l for l in range(28)], Q=Q, layer_heads=layer_heads)
            general_att_maps = get_attention_map(model_general_output, input_general_ids, layers=[l for l in range(28)], Q=Q, layer_heads=None)

        att_map_list = {}
        for layer in att_maps.keys():
            att_map = att_maps[layer]
            if norm:
                try:
                    general_att_map = general_att_maps[layer]
                except:
                    general_att_map = general_att_maps[layer[0]]
                att_map = att_map / general_att_map

            att_map_list[layer] = {
                'att_map': att_map.tolist()
            }

        qa["input_ids"] = input_ids[0].cpu().tolist()
        qa["attention_maps"] = att_map_list
        qa_set_attention_maps[id] = qa

    os.makedirs("./attention_maps", exist_ok=True)

    np.save(f'./attention_maps/{output_filename}_set_attention_maps{filename_suffix}.npy', qa_set_attention_maps)


In [5]:
import numpy as np

def get_Filtering_KnockOut_acc(json_path, output_filename, filename_suffix, masked_layer, percentile, magnitude_threshold, att_map_layer, layer_heads=None):
    print(f"Loading attention maps from './attention_maps/{output_filename}_set_attention_maps{filename_suffix}.npy'")
    qa_set_attention_maps = np.load(f'./attention_maps/{output_filename}_set_attention_maps{filename_suffix}.npy', allow_pickle=True).item()

    correct = 0
    total = 0
    incorrect_ids_qa_ak=[]

    for id, qa in tqdm(qa_set_attention_maps.items()):
        prompt = f"{qa['question']} Answer the question using a single word or phrase."
        
        # 2nd pass
        input_ids = np.array(qa["input_ids"])
        image_start_token_index = np.where(input_ids==-200)[0][0]
        att_map_list = qa["attention_maps"]

        image_path = os.path.join(data_path, qa['image_name'])

        last_token_index = [idx for idx in range(image_start_token_index + NUM_IMG_TOKENS, input_ids.shape[-1] + NUM_IMG_TOKENS - 1)]

        if not isinstance(last_token_index, list):
            last_token_index = [last_token_index]

        block_config = {}

        if layer_heads is not None:
            att_map = []
            for layer_head in layer_heads:
                layer, head = layer_head[0], layer_head[1]
                att_map.append(att_map_list[(layer, head)]['att_map'])
            att_map = np.array(att_map).mean(axis=0)
        else:
            att_map = np.array(att_map_list[att_map_layer]['att_map'])

        flattened_att_map = att_map.flatten()
        if percentile is not None:
            threshold = np.percentile(flattened_att_map, percentile)
            low_attention_indices = np.where(flattened_att_map < threshold)[0]
        elif magnitude_threshold is not None:
            threshold = magnitude_threshold
            flattened_att_map = normalize(flattened_att_map.copy())
            low_attention_indices = np.where(flattened_att_map < threshold)[0]
            

        image_range = [image_start_token_index + patch_index for patch_index in low_attention_indices]


        block_ids = [image_range, last_token_index]
        temp = [(stok1, stok0) for stok0 in block_ids[0] for stok1 in block_ids[1]]


        for layer in masked_layer:
            block_config[layer] = copy.deepcopy(temp)

        with torch.inference_mode():
            block_attn_hooks = set_block_attn_hooks_llava(bot.model, block_config, block_desc="Image->Question")
            _, _, response_qs = bot.inference(prompt, image_path)
            remove_wrapper_llava(bot.model, block_attn_hooks)
            
        answer = qa['answer'].capitalize()

        clean_response = response_qs.replace("<|endoftext|>", "").capitalize()

        if clean_response in answer or answer in clean_response:
            correct += 1
        else:
            incorrect_ids_qa_ak.append(id)
        total += 1

    print(f"OG Accuracy on {json_path.replace('_', ' ')}: {(correct / total) * 100:.4f}%, {correct}/{total}")

    os.makedirs("./incorrect", exist_ok=True)
    with open(f"./incorrect/incorrect_ids_{output_filename}_ak{filename_suffix}.json", "w") as f:
        json.dump(incorrect_ids_qa_ak, f)

In [6]:
Q = False
norm = True
masked_layer = [16]
percentile = 50
magnitude_threshold = None
att_map_layer = 16
layer_heads = None

layer_heads = [[15, 4], [16, 5], [15, 1], [10, 6], [4, 0], [15, 24], [19, 6], [8, 3], [19, 18], [18, 10], [4, 17], [16, 2], [16, 17], [16, 1], [14, 23], [6, 18], [18, 19], [6, 11], [0, 2], [17, 26]]


filename_suffix = ""
if Q:
    filename_suffix += "_Q"
else:
    filename_suffix += "_Last"
if norm:
    filename_suffix += "_Norm"
else:
    filename_suffix += "_woNorm"
if layer_heads is not None:
    if len(layer_heads) > 6:
        filename_suffix += f"_head_top{len(layer_heads)}"
    else:
        filename_suffix += f"_head{layer_heads}"


data_path = ""
question_file = ""
output_filename = "RAD"

get_Attention_Maps(question_file, output_filename, Q=Q, norm=norm, layer_heads=layer_heads, filename_suffix=filename_suffix)
get_Filtering_KnockOut_acc(question_file, output_filename, filename_suffix, masked_layer=masked_layer, percentile=percentile, magnitude_threshold=magnitude_threshold, att_map_layer=att_map_layer, layer_heads=layer_heads)

Loading attention maps from './attention_maps/RAD_set_attention_maps_Q_Norm.npy'


100%|██████████| 1297/1297 [08:43<00:00,  2.48it/s]


OG Accuracy on /home/user/Desktop/DiskSDA/dataset/VQA RAD/filter/sampled data: 68.6970%, 891/1297
